<a href="https://colab.research.google.com/github/juanigp/CT-denoising/blob/master/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone -l -s git://github.com/juanigp/CT-denoising.git cloned-repo
%cd cloned-repo
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

import os
from IPython.core.debugger import set_trace
from models.EDCNN import EDCNN
from utils import utils
import torch
import torch.nn as nn
import torch.utils.data.sampler as sampler
from torch.autograd import Variable
from matplotlib import pyplot as plt
import random
import numpy as np

Cloning into 'cloned-repo'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 291 (delta 10), reused 0 (delta 0), pack-reused 270
Receiving objects: 100% (291/291), 41.96 MiB | 23.12 MiB/s, done.
Resolving deltas: 100% (145/145), done.
/content/cloned-repo
Mounted at /gdrive


In [0]:
models_dir = r'/gdrive/My Drive/models/'
models_list = os.listdir(models_dir)
models_100 = [os.path.join(models_dir,model) for model in models_list if 'EDCNN_checkpoint' in model]
models_250 = [os.path.join(models_dir,model) for model in models_list if 'EDCNN_250_checkpoint' in model]
models_360 = [os.path.join(models_dir,model) for model in models_list if 'EDCNN_360_checkpoint' in model]

In [26]:
#llevarlo a funcion q tome lista de modelos

#hyperparameters:
num_epochs = 100
batch_size = 32
learning_rate = 0.00001

#instantiating the model:
model = EDCNN()

#loss function
criterion = nn.L1Loss()

#optimizer algorithm
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()

csv_file = r'/gdrive/My Drive/patches/360_FBPPhil.csv' #######################
dataset = utils.CTVolumesDataset(csv_file)

#split of data in training and testing data:
#the .csv is shuffled (using the same seed everytime for repeatability)
num_samples = len(dataset)
total_idx = list(range(num_samples))
random.seed(10)
random.shuffle(total_idx)

#pick 10% of samples to test
testing_samples_percentage = 0.1
split_index = int( num_samples * testing_samples_percentage )
#pick the first 10% of samples in the shuffled dataset for testing
testing_idx = total_idx[0 : split_index]
#pick the other 90% of samples in the shuffled dataset for training
training_idx = total_idx[split_index : num_samples]
#random samplers for training and testing
training_sampler = sampler.SubsetRandomSampler(training_idx)
testing_sampler = sampler.SubsetRandomSampler(testing_idx)
#dataloaders for training and testing
training_dataloader = torch.utils.data.DataLoader(dataset = dataset, batch_size = batch_size, sampler = training_sampler)
testing_dataloader = torch.utils.data.DataLoader(dataset = dataset, batch_size = batch_size, sampler = testing_sampler)

metrics_file_name = 'training_loss_360_.csv' #################################
metrics_file_dir = os.path.join(models_dir, metrics_file_name)

for checkpoint_dir in models_360:##############################################
  checkpoint = torch.load(checkpoint_dir)
  model.load_state_dict(checkpoint['model'])
  model.eval()
  with torch.no_grad():

    #ON TRAINING DATASET
    training_epoch_loss = 0
    num_batches = 0
    for i, (lo_res, hi_res) in enumerate(training_dataloader):
        #add an extra dimension:
        lo_res = utils.var_or_cuda( lo_res.unsqueeze(1) )
        hi_res = utils.var_or_cuda(hi_res)
        if lo_res.size()[0] != batch_size:
          continue  
        num_batches += 1 
        #forward pass 
        outputs = model(lo_res)
        loss = criterion(outputs, hi_res.unsqueeze(1))
        training_epoch_loss += loss.item()
        #backward & optimize
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()

    training_epoch_loss /= num_batches
    print('Done training epoch')

    #ON TESTING DATASET
    testing_epoch_loss = 0
    num_batches = 0
    for batch, (lo_res, hi_res) in enumerate(testing_dataloader):
      #add an extra dimension:
      lo_res = utils.var_or_cuda( lo_res.unsqueeze(1) )
      hi_res = utils.var_or_cuda(hi_res)
      if lo_res.size()[0] != batch_size:
          continue
      num_batches += 1
      outputs = model(lo_res)
      loss = criterion(outputs, hi_res.unsqueeze(1))
      testing_epoch_loss += loss.item()

    testing_epoch_loss /= num_batches
    print('done testing epoch')     

  csv_line = str(training_epoch_loss) + ',' + str(testing_epoch_loss) + '\n'
  with open(metrics_file_dir , 'a+') as file:
      file.write(csv_line)



Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done testing epoch
Done training epoch
done 